# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [3]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [4]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [5]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-02-15'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-USD',start_date, today).reset_index()       # descargar datos de yahoo Finance del activo Bitcoin vs Dolar

btc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2597,2023-02-10,"$21,819.01","$21,941.19","$21,539.39","$21,651.18","$21,651.18",27078406594
2598,2023-02-11,"$21,651.84","$21,891.41","$21,618.45","$21,870.88","$21,870.88",16356226232
2599,2023-02-12,"$21,870.90","$22,060.99","$21,682.83","$21,788.20","$21,788.20",17821046406
2600,2023-02-13,"$21,787.00","$21,898.41","$21,460.09","$21,808.10","$21,808.10",23918742607
2601,2023-02-14,"$21,801.82","$22,293.14","$21,632.39","$22,220.80","$22,220.80",26792596581


In [6]:
# Validamos que no hay datos vacíos

btc_df.isnull().sum()


Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [12]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[["Date", "Open"]]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename(columns=new_names)
df.tail()

,ds,y
2597,2023-02-10,"$21,819.01"
2598,2023-02-11,"$21,651.84"
2599,2023-02-12,"$21,870.90"
2600,2023-02-13,"$21,787.00"
2601,2023-02-14,"$21,801.82"


In [13]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [14]:
df.tail()

,ds,y
2597,2023-02-10,"$21,819.01"
2598,2023-02-11,"$21,651.84"
2599,2023-02-12,"$21,870.90"
2600,2023-02-13,"$21,787.00"
2601,2023-02-14,"$21,801.82"


In [16]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "Serie temporal Bitcoin Precio de Apertura"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [17]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"

m = Prophet(seasonality_mode = "multiplicative")


In [18]:
#TAREA: Entrena el modelo m con el dataframe de datos df

m.fit(df)


INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [19]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2962,2024-02-10
2963,2024-02-11
2964,2024-02-12
2965,2024-02-13
2966,2024-02-14


In [21]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05
...,...
2962,2024-02-10
2963,2024-02-11
2964,2024-02-12
2965,2024-02-13


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [22]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [23]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2962,2024-02-10,"$-17,530.53","$-36,752.39","$2,632.05"
2963,2024-02-11,"$-17,791.16","$-37,023.61","$1,993.14"
2964,2024-02-12,"$-18,035.74","$-37,622.03","$2,295.17"
2965,2024-02-13,"$-18,262.16","$-37,189.00","$2,376.97"
2966,2024-02-14,"$-18,436.69","$-38,227.42","$2,836.20"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [24]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [25]:
plot_components_plotly(m, forecast)